<a href="https://colab.research.google.com/github/JaganMurugan/Learning/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt

In [6]:
import time

"""
print(time.time())

time.sleep(2)

start = time.time()

time.sleep(2)

end = time.time()

print("Elapsed time:", end - start)
"""

1746759254.337808
Elapsed time: 2.0004985332489014


In [2]:
from typing import Iterable, Tuple
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms

In [15]:
class LeNet5(nn.Module):
  def __init__(self):
    super().__init__()

    # convolution layers
    self._body = nn.Sequential(
        # First Convolution Layer
        # input size = (32, 32), output size = (28, 28)
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
        # ReLU activation
        nn.ReLU(inplace=True),
        # Max pool 2-d
        nn.MaxPool2d(kernel_size=2),

        # Second Convolution Layer
        # input size = (14, 14)
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),
        # output size = (5, 5)
    )

    # fully connected layers
    self._head = nn.Sequential(
        # First fully connected layer
        # in_features = total number of weights in last conv layer = 16 * 5* 5
        nn.Linear(in_features=16*5*5, out_features=120),

        # ReLU activation
        nn.ReLU(inplace=True),

        # Second fully connected layer
        # in_features = output of last linear Layer = 120
        nn.Linear(in_features=120, out_features=84),

        # ReLU activation
        nn.ReLU(inplace=True),

        # Third fully connected layer. It is also output layer
        # in_features = output of last linear layer = 84
        # and out_features =number of classes = 10 (MNIST data 0-9)
        nn.Linear(in_features=84, out_features=10)
        )

  def forward(self, x):
    # apply feature extractor
    x = self._body(x)
    # flatten the output of conv layers
    # dimesion should be batch_size * number_of weights_i last_conv layer
    x = x.view(x.size([0], -1))
    # apply classification head
    x = self._head(x)
    return x




In [16]:
lenet5_model = LeNet5()
print(lenet5_model)

LeNet5(
  (_body): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (_head): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [17]:
"""
train_test_transforms = transforms.Compose([
    # Resize to 32X32
    transforms.Resize((32, 32)),
    # this re-scales image tensor values between 0-1. image_tensor /= 255
    transforms.ToTensor(),
    # subtract mean (0.1307) and divide by variance (0.3081)
    transforms.Normalize((0.1307,), (0.3081))
])
"""

def get_data(batch_size, data_root='data', num_workers=1):

  train_test_tranforms = transforms.Compose([
      transforms.Resize(32, 32),
      transforms.ToTensor(),
      transforms.Normalize((0.1307, ), (0.3081, ))
  ])

  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root=data_root, train=True, download=False, transform=train_test_transforms),
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers
  )

  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root=data_root, train=False, download=False,transform=train_test_transforms),
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers
  )

  return train_loader, test_loader



In [18]:
@dataclass
class SystemConfiguration:
  '''
  Describes the common system setting needed for reproducible training
  '''

  seed: int = 42 #seed number to set the state of all random generators
  cudn_benchmark_enabled: bool = True # enable CuDNN benchmark for the sake of performance
  cudnn_deterministic: bool = True # make cudnn deterministic (reproducible training)